# Plotter of pearl trajectory

## Goals of this notebook

Main goal - create a calculator that uses cannon construction and target displacement to calculate number of wind charges

## Sources

Great page with math: https://minecraft.wiki/w/Tutorial:360_degree_ender_pearl_cannon


## Static data

In [ ]:
import numpy as np
from enum import Enum

# Will use Minecraft notation - X is 'right', Y is 'up', Z is 'toward me'

""" 
Trajectory planning is solved. Need to process directions and offsets.

User inputs: item frame facing direction, item frame position, target position. All absolute, world reference frame.

Stages of algorythm:
- Get global position of pearl launch area
    Take default offset, shift to new direction, add to item frame position
- Get relative position of target in global frame
- Rotate relative position into cannon frame
- Get UI quadrant and correct charge pair

"""

# Enum for Minecraft world directions. East is +X, South is +Z.
class cardinalDir(Enum):
    NORTH = 0
    EAST = 1
    SOUTH = 2
    WEST = 3

# Array of wind charge positions relative to launch zone center. 
# Assumes that charge stacks enter in bottom-left corner (from East / positive X). 
# Also assumes that explosion center is at zero relative height (for now)
# Coordinates: +X is left, +Y is vertical up, +Z is forward (away from camera)
chargePositions = np.array([
    [+0.83375, 0.75, +0.16625],      # earliest pos in spiral
    [-0.16625, 0.75, +0.84375], 
    [-0.84375, 0.75, -0.16625], 
    [+0.16625, 0.75, -0.84375]       # latest pos in spiral (gets inside 1st block)
    ])

# Facing direction of cannon in default orientation. 
# Taken by looking at item frame
defaultFacingDir = cardinalDir.WEST

# Offset from block behind item frame to center of 2x2 explosion area
# In default facing direction, expPos = itemFramePos + offset
itemToOriginOffset = np.array([-28, 14, -18])

# Coordinate of ender pearl (feet pos) at the moment of explosion, relative to 2x2 area center. 
# Same orientation as for wind charge coordinates
# (for now assume it's in top-right quadrant)
pearlPosition = np.array([-0.0625, 0.540222006, +0.0625])




## Technical functions
Functions for movement and orientation calculations

In [ ]:
import numpy as np
from scipy.optimize import fsolve

# More readable indexing for launch area quadrants. 
# Directions are in cannon-local frame
class launchQuadId(Enum):
    NE = 0
    SE = 1
    SW = 2
    NW = 3

def rotateCoord(startDir: cardinalDir, endDir: cardinalDir, coordVec: np.ndarray) -> np.ndarray:
    """
    Rotate coordinate by azimuth from starting to final cardinal direction. 
    Example:
        rotateCoord(NORTH, EAST, [1, 0, 0]) => [0, 0, 1]
    TODO - test use on array of vectors (Nx3 matrix)

    Args:
        startDir (cardinalDir): initial facing direction
        endDir (cardinalDir): final facing direction
        coordVec (np.ndarray): 1x3 vector to be rotated

    Returns:
        np.ndarray: 1x3 vector facing new direction
    """
    angleCCW = (endDir.value - startDir.value) * 90
    angleRad = np.deg2rad(angleCCW)
    rotMatrix = np.eye(3)
    rotMatrix[0][0] = rotMatrix[2][2] = np.cos(angleRad)
    rotMatrix[0][2] = np.sin(angleRad)
    rotMatrix[2][0] = -np.sin(angleRad)
    return np.matmul(coordVec, rotMatrix)



# # Only valid for ender pearls due to ticking order
# def projectileTickStep(pos, vel):
#     # acceleration
#     vel += np.array([0.0, -0.03, 0.0])
#     # drag
#     vel *= 0.99
#     # position update
#     pos += vel
#     return (pos, vel)

def pearlInitVelFromEndPos(tick: int, displacement: np.ndarray) -> np.ndarray:
    """
    Get initial velocity to get required ender pearl displacement after some time
    (see wiki for formula)

    Args:
        tick (int): flight time from trajectory start in gameticks
        displacement (np.ndarray): target displacement as 1x3 vector

    Returns:
        np.ndarray: initial velocity as 1x3 vector
    """
    
    d = 0.9900000095367432
    a = np.array([0.0, -0.03, 0.0])
    return (1 - d) / (d * (1 - pow(d, tick))) * (displacement - d * tick * a / (1 - d)) + (d / (1 - d) * a)

def pearlPosFromInitVel(tick: float, vel0: np.ndarray) -> np.ndarray:
    """
    Get ender pearl displacement from initial velocity after some time
    (see wiki for formula)

    Args:
        tick (float): time from trajectory start in gameticks
        vel0 (np.ndarray): initial velocity as 1x3 vector

    Returns:
        np.ndarray: displacement relative to starting point as 1x3 vector
    """

    d = 0.9900000095367432
    a = np.array([0.0, -0.03, 0.0])
    return (d * (1 - pow(d, tick))) / (1 - d) * (vel0 - d / (1 - d) * a) + (d * tick * a / (1 - d))

def getExplosionMatrix(velA:np.ndarray, velB:np.ndarray) -> np.ndarray:
    """
    Get E matrix from two 3D explosion velocity vectors
    (see tutorial wiki page)

    Args:
        velA (np.ndarray): velocity from first explosion as 1x3 vector
        velB (np.ndarray): velocity from second explosion as 1x3 vector

    Returns:
        np.ndarray: 2x3 E matrix
    """
    
    velMatrix = np.transpose(np.vstack((velA, velB)))
    A = np.matmul(np.transpose(velMatrix), velMatrix)
    B = np.linalg.inv(A)
    C = np.matmul(B, np.transpose(velMatrix))
    return C

def getChargePearlPushVelocity(explosionCenterPos: np.ndarray, pearlFeetPos: np.ndarray) -> np.ndarray:
    """
    Calculate impulse to ender pearl from one wind charge explosion. 
    Assumes 100% exposure 
    (see Explosion wiki page for formulas)

    Args:
        explosionCenterPos (np.ndarray): position of wind charge explosion as 1x3 vector
        pearlFeetPos (np.ndarray): position of ender pearl feet (reported position) as 1x3 vector

    Returns:
        np.ndarray: velocity change as 1x3 vector (away from wind charge)
    """
    
    feetDist = pearlFeetPos - explosionCenterPos
    PearlEyeHeight = 0.2125
    WindChargePower = 3.0
    exposure = 1.0    
    magnitude = (1 - np.linalg.norm(feetDist) / (2 * WindChargePower)) * exposure
    eyeVector = feetDist + np.array([0.0, PearlEyeHeight, 0.0])
    velocityVector = eyeVector / np.linalg.norm(eyeVector) * magnitude
    return velocityVector

def findChargeAmount(velA: np.ndarray, velB: np.ndarray, targetOffs: np.ndarray) -> np.ndarray:
    """
    Solve for wind charge stack sizes and flight time

    Args:
        velA (np.ndarray): velocity added to ender pearl from one wind charge in position A. Format - 1x3 vector
        velB (np.ndarray): velocity added to ender pearl from one wind charge in position B. Format - 1x3 vector
        targetOffs (np.ndarray): relative offset from initial pearl position to target

    Returns:
        np.ndarray: nearest solution as 1x3 vector of floats. 
            Element 0 - flight time (in gameticks). Element 1 - size of charge stack A. Element 2 - size of charge stack B.
    """
    
    # idea - use gameticks as another variable, 3 total. Return full pos error
    def optimizeFunc(x):
        tick, amtA, amtB = x
        velVector = amtA * velA + amtB * velB
        pearlPos = pearlPosFromInitVel(tick, velVector)
        posError = targetOffs - pearlPos
        return posError
    result = fsolve(optimizeFunc, [1, 0, 0])
    
    return result # type: ignore

def getLocalQuadData(targetOffs : np.ndarray) -> tuple[launchQuadId, np.ndarray, np.ndarray]:
    """
    Get shooting quadrant info from target position offset

    Args:
        targetOffs (np.ndarray): target position relative to acceleration zone in cannon-local frame

    Returns:
        tuple[launchQuadId, np.ndarray, np.ndarray]: id of quad for UI, position of first charge stack, position of last charge stack
    """
    
    # Tested, adjusted, seems to give correct results.
    
    chargePearlVectors = chargePositions - pearlPosition
    
    # Add target displacement into same array for batch processing
    allVectorArr = np.vstack((chargePearlVectors, -targetOffs))
    chargeAngles = np.arctan2(-allVectorArr[:, 2], allVectorArr[:, 0])
    simpleAnglesDeg = np.mod(np.rad2deg(chargeAngles) + 360, 360)
    targetAngleDeg = simpleAnglesDeg[4]
    
    # Find nearest angle in CCW direction
    ccwNearestIndex = 0
    # Get all angles bigger (more CCW) than target angle
    cwAngles = [(i, ang) for i, ang in enumerate(simpleAnglesDeg[0:4]) if ang > targetAngleDeg]
    # Get nearest CCW angle to target
    if len(cwAngles) == 0:
        ccwNearestIndex = 0
    else:
        ccwNearestIndex = min(cwAngles, key=lambda x: x[1])[0]
    
    cwNearestIndex = (ccwNearestIndex + 1) % 4
    quadId = launchQuadId(cwNearestIndex)
    firstStackCoord = chargePositions[max(cwNearestIndex, ccwNearestIndex)]
    secondStackcoord = chargePositions[min(cwNearestIndex, ccwNearestIndex)]
    
    return (quadId, firstStackCoord, secondStackcoord)


## Testing area

In [14]:
import numpy as np



# Support functions for orientation adjustments

"""
Testing - set target to [10000, 55, 10000]. 

Program output:
Target offset: [10109.   -43. 10107.]; Real offset: [10117.50958899   -43.78444942 10137.38472755]
Quad ID: 3; Flight time: 97.0; Charges A: 133.0; Charges B: 232.0
Initial predicted velocity: [164.1025045    0.99255313 164.42487237]

Teleport happened on coordinates: 10131, 56, 9188.

Record of first frame of movement: 
>Pos:[55.37481754438798d,99.53036276023064d,42.26555268852343d]
Vel:[164.43731754438798d,0.9901407544247491d,149.26555268852343d]

Ok. Some issue with Z velocity calculation?
X velocity is close enough. Wrong count of charges would've influenced both axis
Maybe i got distance constants wrong? Or did some math wrong?

Found an issue - pearl was aligning to 0.0 on 1st (Z) axis, likely due to anvil foot. 
Tried repeating this - too unreliable, can't get 2nd axis to align. 
Went back to 2 anvils, this seems to work always.

Let's test again...
Ok, now landed at 9946, 56, 9968. Pretty good.
Maybe errors are mainly due to height - i set 55, real is 56 plus some error. 
Pearl is still coming pretty shallow, at 10s blocks per second

"""

"""
Test 2 - created a copy rotated 90 deg and way higher.
Time to debug orientation handling.

error 1 - firing origin pos is 54 x -46, code thinks it's at 53 x -46. 
    For now adjusted offset to -18 from -17
    TODO - see if this is my error or issue with coord reporting

Quad ID was chosen correctly

Bug - i think stack order is flipped when in quad 0. 
    Need to push "second" stack first and "first" stack last. 
    For now this is Errata. Later need to fix subtractor and this program.

Stacks are sorted correctly even on quad 0 (i.e. 1st stack is correctly sized?)

Fixed subtractor. I think program works too.



"""

"""
Small test on rotated cannon
Target was [-1000, 56, -1000]. Landed at [-999, 56, -977].

Program output for this setting:
Target offset: [-954. -209. 1054.]; Real offset: [-930.09540176 -208.92501021 1052.43300151]
Quad ID: 0; Flight time: 149.0; Charges A: 18.0; Charges B: 10.0
Initial predicted velocity: [-12.10197194   0.06956296  13.6937723 ]

First frame of movement:
>Pos:[40.38066528960899d,265.57570006559496d,-58.04345233465931d]
Vel:[-13.556834710391009d,0.035478059789167696d,-11.980952334659305d]

Caution - velocity and offsets are in cannon frame
With compensating for this i think everything works out good.
Yes. Confirmed in [-150 56 -1500] test - not counting quad 0 all works ok.

"""



def calculateLaunchParameters(cannonDir : cardinalDir, cannonPos: np.ndarray, targetPos: np.ndarray):
    # return - quad ID, flight time, charges A and B, actual position    

    # TODO - move required data into global ref frame, finish collecting output

    # frameFacingDir = cardinalDir.WEST
    # frameBlockPos = np.array([36, 251, -18])
    # targetPos = np.array([-150, 56, -1500])

    shootingPos = cannonPos + rotateCoord(defaultFacingDir, cannonDir, itemToOriginOffset)
    globalTargetOffset = targetPos - shootingPos
    localTargetOffset = rotateCoord(cannonDir, defaultFacingDir, globalTargetOffset)


    quadId, stackPosA, stackPosB = getLocalQuadData(localTargetOffset)
    velA = getChargePearlPushVelocity(stackPosA, pearlPosition)
    velB = getChargePearlPushVelocity(stackPosB, pearlPosition)
    idealCannonSetting = findChargeAmount(velA, velB, localTargetOffset)
    # Get realistic values, check rounding error
    ticks, chargesA, chargesB = idealCannonSetting.round()
    velVector = chargesA * velA + chargesB * velB
    actualLocalOffs = pearlPosFromInitVel(ticks, velVector)
    actualLandingPos = shootingPos + rotateCoord(defaultFacingDir, cannonDir, actualLocalOffs)
    
    return (quadId, ticks, chargesA, chargesB, actualLandingPos)

frameFacingDir = cardinalDir.EAST
frameBlockPos = np.array([100, 100, -200])
targetPos = np.array([-1000, 50, -1000])
qid, time, chgA, chgB, realPos = calculateLaunchParameters(frameFacingDir, frameBlockPos, targetPos)

print(f"Quad ID: {qid.value}; Flight time: {time}; Charges A: {chgA}; Charges B: {chgB}")
print(f"Real end position: {realPos}")


Quad ID: 3; Flight time: 77.0; Charges A: 10.0; Charges B: 28.0
Real end position: [-994.79180319   49.32645322 -977.47270705]
